In [8]:
import pygame as p, random as r, time, numpy as np
import neat
p.init()
d_w = 800            # Display width
d_h = 600            # Display height
gap = 200            # Size of gap bird flies through
distance = 300       # Distance between pipes. higher value = lower distance
speed = 8            # Speed the bird moves horizontally
jump_height = 19     # How much upwards velocity the bird gets upon jumping
gravity = 1.25       # How fast upwards velocity decreases

In [9]:
class Bird():
    def __init__(self, filename):
        self.image = p.image.load(filename) 
        self.image = p.transform.scale(self.image, (60, 42))
        self.orig_image = p.image.load(filename)
        self.orig_image = p.transform.scale(self.orig_image, (60, 42))
        self.vel = 0
        self.dead = False
        self.y = 400
        self.x = 20
        
    def flap(self):
        self.vel = jump_height
        
    def update(self):
        # Updates game state every time step
        self.image = p.transform.rotate(self.orig_image, self.vel*3) # Rotate bird slightly when he jumps
        self.y -= self.vel
        self.vel -= gravity
        if self.y <= 0:
            self.y = 0
            
class Pipe():
    def __init__(self):
        self.top = r.randint(50, d_h-gap-150) # bottom of top of pipe y value
        self.bottom = self.top + gap          # top of bottom of pipe y value
        self.bottom_image = p.surfarray.make_surface(np.zeros((66, d_h-self.bottom)))
        self.top_image = p.surfarray.make_surface(np.zeros((66, self.top)))
        self.x = d_w
        
    def update(self, bird_x, bird_y):
        # Updates game state every time step. Returns whether bird is dead
        self.x -= speed # Makes pipe move left
        if ((bird_y + 5 < self.top or bird_y + 42 > self.bottom) and abs(bird_x + 30 - self.x - 33) < (30+33)) or bird_y >= d_h: # Checks if bird is in contact with pipe. Returns True if bird should die.
            return True
        return False

    

In [10]:
'''
Run the game. Press W to jump. Press S to turn on slow motion.
'''
# Setting up display
clock = p.time.Clock()
gameDisplay = p.display.set_mode((d_w, d_h), )
p.display.set_caption('Flappy Bird')
background = p.image.load('background.png')
background = p.transform.scale(background, (d_w, d_h))

exit = False
while not exit:
    #Setting up
    bird = Bird('flapbird.png')
    pipes = [Pipe()]
    started = False # If game has started
    slow_mo = False # Runs game at lower FPS
    while not bird.dead:
        if started:
            bird.update()
            
            for pipe in pipes[-2:]: # Checks if bird is touching a pipe
                if pipe.update(bird.x, bird.y):
                    bird.dead = True
                    break
        
        # Checks if any buttons are pressed
        for event in p.event.get():
            if event.type == p.QUIT:
                bird.dead = True
                exit = True
                break
                
            if event.type == p.KEYDOWN:
                if event.key == p.K_q:
                    bird.dead = True
                    exit = True
                    break
                    
                if event.key == p.K_w:
                    bird.flap()
                    started = True
                
                if event.key == p.K_s:
                    slow_mo = not slow_mo
        
        #Adds bird and background to image
        gameDisplay.blit(background, (0,0))
        gameDisplay.blit(bird.image, (bird.x, bird.y))
        
        pipes = pipes[-2:] # Removes old pipes
        if pipes[-1].x <= distance: # Adds new pipes if necessary
            pipes.append(Pipe())
            
        # Adds pipes to image
        for pipe in pipes:
            gameDisplay.blit(pipe.bottom_image, (pipe.x, pipe.bottom))
            gameDisplay.blit(pipe.top_image, (pipe.x, 0))

        p.display.update()
        if slow_mo:
            clock.tick(20)
        else:
            clock.tick(60)
        
p.quit()